In [0]:
# Best result plot
# with k-Fold
# poly degree = 4,5
 
# Import necessary libraries
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVR 
from sklearn.metrics import mean_absolute_error # absolute: necessary to assess accuracy 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

#Write results fucntion
def writeResults(predictions):
    predictions = predictions.astype(np.int64)
    results = pd.read_csv("submission_format.csv")
    results["total_cases"] = predictions
    results.to_csv("Newsubmition.csv", index=False)

X = pd.read_csv("dengue_features_train.csv.csv")
y = pd.read_csv("dengue_labels_train.csv")
y = y["total_cases"]

degrees=[4,5]

# Normalization
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# k-fold
kf = KFold(n_splits=5)
for d_value in degrees:
  maes = []
  for train_index, test_index in kf.split(X, y):
    svr = SVR(kernel="poly", degree=d_value, gamma='scale', C=0.5, epsilon=0.1)
    print(svr)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Model
    svr.fit(X_train, y_train)
    predictions = svr.predict(X_test)
    mae = mean_absolute_error(y_test,predictions)
    maes.append(mae)
    print("Error: ",mae)
  plt.plot(range(y_test.shape[0]),predictions,label=str("Poly. degree=" + str(d_value)))
  print("Mean average Error", np.mean(maes))
#print(predictions)
plt.plot(range(y_test.shape[0]),y_test,label="test data")
plt.xlabel("Week of the year")
plt.ylabel("Number of dengue cases")
plt.legend()
plt.savefig("out.png")

# Apply model on test set and make predictions
X_testset = pd.read_csv("new_dengue_features_test.csv")
X_testset = scaler.fit_transform(X_testset)

pred_test = svr.predict(X_testset)
#print(pred_test)
writeResults(pred_test)



